In [ ]:
# Data :
#     +) https://www.kaggle.com/datasets/vfomenko/young-affectnet-hq
#     +) https://www.kaggle.com/datasets/arnabkumarroy02/ferplus

In [ ]:
# ============================================================
# 1. STANDARD LIBRARIES (Thư viện chuẩn Python)
# ============================================================
import os
import json
import random
from glob import glob
from collections import Counter
from io import BytesIO

# ============================================================
# 2. DATA SCIENCE & UTILITIES (Xử lý dữ liệu, ảnh, request)
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm

# Scikit-learn (Metrics & Model Selection)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ============================================================
# 3. PYTORCH & DEEP LEARNING
# ============================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.amp as amp  # Mixed Precision Training

# Data handling
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

# Torchvision
from torchvision import transforms

# Schedulers
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR

# Data processing

In [ ]:
# ============================================================
# 1) CẤU HÌNH (CONFIG)
# ============================================================

# Danh sách các nhãn (class) hợp lệ
VALID_CLASSES = ["angry", "fear", "happy", "neutral", "sad", "surprise"]

# Bản đồ ánh xạ các nhãn khác nhau về nhãn chuẩn
EMOTION_MAP = {
    "anger": "angry",
    "disgust": "angry",   # Gom 'ghê tởm' vào 'giận dữ'
    "contempt": "angry",  # Gom 'khinh bỉ' vào 'giận dữ'
    "fear": "fear",
    "happiness": "happy",
    "sadness": "sad",
    "surprise": "surprise",
    "neutral": "neutral",
}


# ============================================================
# 2) LÀM SẠCH NHÃN (CLEAN LABELS)
# ============================================================

def clean_labels(df):
    """
    Chuẩn hóa tên nhãn về chữ thường, xóa khoảng trắng thừa,
    ánh xạ về tên chuẩn và lọc bỏ các nhãn không nằm trong VALID_CLASSES.
    """
    df['label'] = (
        df['label']
        .str.lower()
        .str.strip()
        .replace({
            'suprise': 'surprise',  # Sửa lỗi chính tả
            'happiness': 'happy',
            'sadness': 'sad',
            'anger': 'angry',
            'disgust': 'angry',
            'contempt': 'angry',
        })
    )
    # Chỉ giữ lại các dòng có nhãn nằm trong danh sách cho phép
    return df[df['label'].isin(VALID_CLASSES)].reset_index(drop=True)


# ============================================================
# 3) TẢI DỮ LIỆU (LOAD DATASET)
# ============================================================

def load_dataset(base_dir, exts=("jpg", "jpeg", "png")):
    """
    Quét thư mục để tìm ảnh. Tự động phát hiện cấu trúc thư mục.
    Hỗ trợ cấu trúc: root/train/label/image.jpg hoặc root/label/image.jpg
    """
    rows = []
    # Kiểm tra xem thư mục gốc có sub-folder 'train'/'test'
    has_subset = any(x in os.listdir(base_dir) for x in ["train", "test"])

    folders = ["train", "test"] if has_subset else [""]

    for subset in folders:
        subset_dir = os.path.join(base_dir, subset)
        # Nếu thư mục không tồn tại thì bỏ qua
        if not os.path.exists(subset_dir):
            continue
            
        for label in os.listdir(subset_dir):
            label_dir = os.path.join(subset_dir, label)
            if not os.path.isdir(label_dir):
                continue
            
            # Lấy tất cả các đuôi file ảnh (jpg, png...)
            for ext in exts:
                # Dùng glob để tìm đường dẫn file
                for path in glob(os.path.join(label_dir, f"*.{ext}")):
                    rows.append({
                        "image_path": path, 
                        "label": label, 
                        "subset": subset if subset else None # Lưu thông tin train/test gốc nếu có
                    })

    return pd.DataFrame(rows)


# ============================================================
# 4) GỘP DỮ LIỆU VÀ CÂN BẰNG (MERGE)
# ============================================================

def merge_datasets(dataset_cfgs, test_size=0.15, random_state=42):
    """
    Load và gộp dữ liệu. Giữ nguyên tỉ lệ mất cân bằng (imbalanced) 
    để xử lý bằng Class Weight trong Loss Function.
    """
    merged = []
    print("--- Bắt đầu gộp dữ liệu ---")

    for cfg in dataset_cfgs:
        # 1. Load thô
        df = load_dataset(cfg["path"])
        if df.empty: continue

        # 2. Chuẩn hóa nhãn (Map -> Lower -> Clean)
        df['label'] = df['label'].str.lower().str.strip()
        if "label_map" in cfg:
            df['label'] = df['label'].map(cfg["label_map"]).fillna(df['label'])
        df = clean_labels(df)

        # 3. Lọc subset (nếu chỉ lấy tập train của dataset nguồn)
        if cfg.get("has_subset", False) and "subset" in df.columns:
            df = df[df["subset"] == "train"]

        merged.append(df)
        print(f" -> Đã thêm {len(df)} ảnh từ {cfg['path']}")

    if not merged:
        raise ValueError("Không tìm thấy dữ liệu hợp lệ nào!")

    # 4. Gộp tất cả và Xáo trộn
    df_all = pd.concat(merged, ignore_index=True)
    
    # 5. Chia Train/Val (Stratify để tập Val có tỉ lệ giống tập Train)
    train_df, val_df = train_test_split(
        df_all,
        test_size=test_size,
        stratify=df_all['label'],
        random_state=random_state
    )

    print(f"\nTổng ảnh: {len(df_all)} (Train: {len(train_df)}, Val: {len(val_df)})")
    print("Phân bố Train (Chưa cân bằng):", train_df['label'].value_counts().to_dict())

    return train_df.reset_index(drop=True), val_df.reset_index(drop=True)

# ============================================================
# 5) DATA AUGMENTATION (TRANSFORMS)
# ============================================================

def get_transforms(img_size=112):
    """
    Định nghĩa các phép biến đổi ảnh để làm phong phú dữ liệu (Augmentation).
    """
    train_t = transforms.Compose([
        transforms.Grayscale(1),                    # Chuyển về ảnh xám (1 kênh màu)
        transforms.Resize(img_size + 16),           # Resize lớn hơn một chút
        transforms.RandomResizedCrop(img_size, scale=(0.80, 1.0)), # Crop ngẫu nhiên
        transforms.RandomHorizontalFlip(p=0.5),     # Lật ngang ảnh (gương)
        transforms.RandomRotation(15),              # Xoay nhẹ tối đa 15 độ
        transforms.ColorJitter(brightness=0.3, contrast=0.3), # Thay đổi độ sáng/tương phản
        transforms.GaussianBlur(3),                 # Làm mờ nhẹ (giảm nhiễu hạt)
        transforms.ToTensor(),                      # Chuyển sang Tensor và đưa về [0, 1]
        transforms.Normalize([0.5], [0.5]),         # Chuẩn hóa về [-1, 1]
        transforms.RandomErasing(p=0.4, scale=(0.02, 0.2)), # Xóa ngẫu nhiên 1 vùng nhỏ (tránh overfitting)
    ])

    val_t = transforms.Compose([
        transforms.Grayscale(1),
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),            # Chỉ lấy phần trung tâm
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])

    return train_t, val_t


# ============================================================
# 6) DATASET CLASS (PYTORCH)
# ============================================================

class EmotionDataset(Dataset):
    def __init__(self, df, transform, preload=False):
        self.df = df.reset_index(drop=True)
        self.transform = transform
        
        # Tạo từ điển map từ Label (str) -> Index (int)
        self.label_to_idx = {l: i for i, l in enumerate(VALID_CLASSES)}
        self.idx_to_label = {i: l for l, i in self.label_to_idx.items()}
        self.num_classes = len(VALID_CLASSES)

        # Preload: Load toàn bộ ảnh vào RAM
        self.preload = preload
        self.cache = None
        if preload:
            print("Đang load toàn bộ ảnh vào RAM...")
            self.cache = [Image.open(p).convert('RGB') for p in tqdm(df['image_path'])]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Lấy ảnh từ cache hoặc đọc từ đĩa
        if self.cache is not None:
            img = self.cache[idx]
        else:
            img = Image.open(row['image_path']).convert('RGB')

        # Áp dụng transform (augmentation)
        img = self.transform(img)

        # Chuyển nhãn thành số
        label = self.label_to_idx[row['label']]
        
        return img, torch.tensor(label, dtype=torch.long)

    # Hàm tính trọng số cho Loss Function (dùng khi dữ liệu mất cân bằng)
    def get_class_weights(self):
        counts = Counter([self.label_to_idx[l] for l in self.df['label']])
        total = len(self.df)
        # Công thức: weight = tổng / số lượng của class đó
        weights = [total / counts[i] for i in range(self.num_classes)]
        return torch.tensor(weights, dtype=torch.float)

    # Hàm tạo Sampler (để bốc mẫu theo trọng số khi train)
    def get_sampler(self):
        counts = Counter([self.label_to_idx[l] for l in self.df['label']])
        # Trọng số cho từng mẫu = 1 / số lượng class của mẫu đó
        weights = [1.0 / counts[self.label_to_idx[l]] for l in self.df['label']]
        return WeightedRandomSampler(weights, len(weights), replacement=True)


# ============================================================
# 7) CHƯƠNG TRÌNH CHÍNH (MAIN)
# ============================================================

if __name__ == "__main__":
    # Cấu hình các folder dữ liệu
    configs = [
        # Dataset FERPlus, có sẵn folder train/test
        {"path": "/kaggle/input/ferplus", "has_subset": True}, 
        # Dataset AffectNet, cần map lại nhãn
        {"path": "/kaggle/input/young-affectnet-hq", "label_map": EMOTION_MAP}, 
    ]

    # 1. Gộp và xử lý dữ liệu
    train_df, val_df = merge_datasets(configs, test_size=0.1)

    # 2. Lấy các phép transform
    train_t, val_t = get_transforms(img_size=112)

    # 3. Tạo Dataset
    train_ds = EmotionDataset(train_df, train_t, preload=False)
    val_ds = EmotionDataset(val_df, val_t, preload=False)

    # 4. Tạo DataLoader
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

    print("\nDataset đã sẵn sàng!")
    print(f"Train samples: {len(train_ds)}")
    print(f"Val samples:   {len(val_ds)}")

    # Lấy danh sách tên class
    class_names = list(train_ds.label_to_idx.keys()) 
    class_weights = train_ds.get_class_weights()
    print(f"Classes: {class_names}")
    
    # In thử 1 batch để kiểm tra shape
    imgs, labels = next(iter(train_loader))
    print(f"Image batch shape: {imgs.shape}") # Kỳ vọng: [64, 1, 112, 112] (1 kênh màu do Grayscale)
    print(f"Label batch shape: {labels.shape}")

In [ ]:
# Kiểm tra mapping nhãn
print(train_df['label'].value_counts())

# Kiểm tra trùng ảnh giữa train/val
print(len(set(train_df['image_path']) & set(val_df['image_path'])))

# Kiểm tra mapping label->idx
print(train_ds.label_to_idx)

# Kiểm tra shape ảnh đầu vào
imgs, _ = next(iter(train_loader))
print(imgs.shape)


In [ ]:
def plot_distribution(df, title):
    counts = df['label'].value_counts().sort_index()
    plt.figure(figsize=(8,4))
    counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel("Emotion label")
    plt.ylabel("Count")
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.show()

plot_distribution(train_df, "Train Set Class Distribution")
plot_distribution(val_df, "Validation Set Class Distribution")

# Model structure

In [ ]:
# ============================================================
# 1. SE BLOCK (Squeeze-and-Excitation)
# ============================================================
class SEBlock(nn.Module):
    """
    SE Block giúp mô hình tập trung vào các đặc trưng quan trọng.
    Cơ chế: "Nhìn" toàn bộ thông tin (Global Pooling) -> Tính trọng số cho từng kênh (FC layers).
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        # Squeeze: Nén không gian (H, W) thành 1 điểm (1x1) để lấy thông tin tổng quát
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        # Excitation: Mạng nơ-ron nhỏ để học trọng số của các kênh
        self.fc = nn.Sequential(
            # Giảm số kênh xuống để tiết kiệm tham số
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            # Khôi phục lại số kênh ban đầu
            nn.Linear(channels // reduction, channels, bias=False),
            # Sigmoid đưa giá trị về khoảng [0, 1] (tương ứng mức độ quan trọng)
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        # y shape: [Batch, Channel]
        y = self.avg_pool(x).view(b, c) 
        # y shape: [Batch, Channel, 1, 1]
        y = self.fc(y).view(b, c, 1, 1)
        
        # Nhân trọng số (y) vào input gốc (x).
        # Kênh nào quan trọng sẽ được giữ lại/khuếch đại, kênh nhiễu bị giảm đi.
        return x * y


# ============================================================
# 2. BASIC BLOCK + SE (Residual Block)
# ============================================================
class BasicBlockSE(nn.Module):
    """
    Khối ResNet cơ bản có tích hợp SE Block và Dropout.
    Cấu trúc: Conv -> BN -> ReLU -> Conv -> BN -> Dropout -> SE -> Add Residual -> ReLU
    """
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, reduction=16, drop_prob=0.0):
        super().__init__()

        # Conv 1: Có thể giảm kích thước ảnh nếu stride > 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        # Conv 2: Giữ nguyên kích thước
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Tích hợp SE Block
        self.se = SEBlock(out_channels, reduction=reduction)
        
        # Dropout xác suất thấp để tránh overfitting trên tập dữ liệu nhỏ
        self.drop_prob = drop_prob

        # Shortcut (đường tắt): Dùng để cộng input vào output
        self.shortcut = nn.Sequential()
        # Nếu kích thước input khác output (do stride hoặc số kênh thay đổi),
        # cần dùng Conv 1x1 để biến đổi input cho khớp shape để cộng được.
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x) # Lưu lại input gốc

        # Luồng chính
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)

        # Dropout (Regularization): Ngẫu nhiên tắt một số feature map khi train
        if self.training and self.drop_prob > 0:
            out = F.dropout2d(out, p=self.drop_prob, training=True)

        # Áp dụng SE Attention
        out = self.se(out)

        # Cộng đường tắt (Residual Connection)
        out += identity
        out = F.relu(out, inplace=True)
        
        return out


# ============================================================
# 3. MAIN MODEL (EmotionResNet)
# ============================================================
class EmotionResNet(nn.Module):
    """
    Mạng ResNet tùy chỉnh:
    - Input: 1 kênh (ảnh xám)
    - Số kênh bắt đầu: 32 (nhỏ hơn ResNet gốc là 64)
    - Số lớp: Tùy chỉnh (Stem + 4 Layer blocks + Classifier)
    """
    def __init__(self, num_classes=6, reduction=8, dropout=0.3, drop_prob=0.05):
        super().__init__()

        # Stem: Lớp xử lý đầu vào (Input 1 channel -> 32 channels)
        self.stem = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        
        # Các khối Residual (Layer 1 -> 4)
        # layer1: 32 -> 32 channels (giữ nguyên size)
        self.layer1 = self._make_layer(32, 32, blocks=2, stride=1, reduction=reduction, drop_prob=drop_prob)
        # layer2: 32 -> 64 channels (giảm size /2)
        self.layer2 = self._make_layer(32, 64, blocks=2, stride=2, reduction=reduction, drop_prob=drop_prob)
        # layer3: 64 -> 128 channels (giảm size /2)
        self.layer3 = self._make_layer(64, 128, blocks=1, stride=2, reduction=reduction, drop_prob=drop_prob) 
        # layer4: 128 -> 256 channels (giảm size /2)
        self.layer4 = self._make_layer(128, 256, blocks=2, stride=2, reduction=reduction, drop_prob=drop_prob)

        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),          # Dropout ở lớp cuối cực quan trọng
            nn.Linear(256, num_classes)   # Output ra số lớp cảm xúc
        )

    def _make_layer(self, in_c, out_c, blocks, stride, reduction, drop_prob):
        layers = []
        # Block đầu tiên của layer chịu trách nhiệm thay đổi stride/channel
        layers.append(BasicBlockSE(in_c, out_c, stride, reduction, drop_prob))
        
        # Các block tiếp theo giữ nguyên stride=1 và channel
        for _ in range(1, blocks):
            layers.append(BasicBlockSE(out_c, out_c, 1, reduction, drop_prob))
            
        return nn.Sequential(*layers)

    def forward(self, x):
        # Trích xuất đặc trưng (Feature Extraction)
        x = self.stem(x)      # [B, 32, H, W]
        x = self.layer1(x)    # [B, 32, H, W]
        x = self.layer2(x)    # [B, 64, H/2, W/2]
        x = self.layer3(x)    # [B, 128, H/4, W/4]
        x = self.layer4(x)    # [B, 256, H/8, W/8]

        # Phân loại (Classification)
        x = self.avgpool(x)   # [B, 256, 1, 1]
        x = x.view(x.size(0), -1) # Flatten -> [B, 256]
        x = self.classifier(x)    # [B, num_classes]
        return x


# ============================================================
# TEST
# ============================================================
if __name__ == "__main__":
    # Giả lập input: Batch=1, Channel=1 (Gray), Size=112x112
    dummy_input = torch.randn(1, 1, 112, 112)
    
    model = EmotionResNet(num_classes=7)
    
    # Tính tổng số tham số (Parameters)
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Chạy thử forward pass
    output = model(dummy_input)
    
    print("-" * 30)
    print(f"Model: EmotionResNet")
    print(f"Total Params: {total_params / 1e6:.2f}M (Triệu tham số)")
    print(f"Output Shape: {output.shape}")
    print("-" * 30)

# Trainning - Evaluate

In [ ]:
# ============================================================
# 1. CÁC HÀM TÍNH TOÁN CHỈ SỐ (METRICS)
# ============================================================

def compute_metrics(y_true, y_pred, labels=None):
    """
    Tính độ chính xác (Accuracy) và ma trận nhầm lẫn (Confusion Matrix).
    """
    acc = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    return acc, cm


# ============================================================
# 2. HÀM HUẤN LUYỆN VÀ ĐÁNH GIÁ (MAIN LOOP)
# ============================================================

def train_and_evaluate(
    in_dir="input",
    out_dir="checkpoints",
    history_dir="history",
    img_size=48,
    epochs=30,
    lr=1e-3,
    weight_decay=1e-4,
    device=None,
    label_smoothing=0.05,
    early_stop_patience=20
):
    
    # --- 1. Thiết lập thư mục và đường dẫn lưu model ---
    os.makedirs(out_dir, exist_ok=True)
    ckpt_path = os.path.join(out_dir, "best_emotion.pth")
    final_ckpt_path = os.path.join(out_dir, "final_emotion.pth")
    history_path = os.path.join(out_dir, "history.json")

    # --- 2. Cấu hình thiết bị (Device) ---
    # Ưu tiên GPU nếu có, fallback về CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device
    device_type = "cuda" if device.type == "cuda" else "cpu"
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"Using device: {device}")
    if device.type == "cuda":
        print(f"GPU Name: {torch.cuda.get_device_name(0)}")

    # --- 3. Chuẩn bị Data Transforms ---
    train_t, val_t = get_transforms(img_size)

    # --- 4. Khởi tạo Model ---
    model = EmotionResNet(num_classes=len(train_ds.label_to_idx))
    
    # Hỗ trợ chạy trên nhiều GPU (Multi-GPU)
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs via DataParallel")
        model = nn.DataParallel(model)

    model = model.to(device)
    
    # --- 5. Cấu hình Loss, Optimizer, Scheduler ---
    # Lấy class weights để xử lý mất cân bằng dữ liệu
    class_weights = train_ds.get_class_weights().to(device)
    
    # Hàm loss có label smoothing và trọng số lớp
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing, weight=class_weights)
    
    # Optimizer AdamW
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    # Scheduler: Kết hợp Warmup (tăng dần LR) và Cosine Annealing (giảm dần LR)
    warmup_epochs = max(1, int(epochs * 0.05))
    warmup = LinearLR(optimizer, start_factor=0.2, total_iters=warmup_epochs)
    cosine = CosineAnnealingLR(optimizer, T_max=max(1, epochs - warmup_epochs))
    scheduler = SequentialLR(optimizer, schedulers=[warmup, cosine], milestones=[warmup_epochs])
    
    # Scaler cho Mixed Precision Training (FP16)
    scaler = amp.GradScaler(enabled=(device.type == "cuda"))

    # --- 6. Logic Resume (Khôi phục training từ checkpoint) ---
    best_val, start_epoch, best_epoch, no_improve = 0.0, 0, 0, 0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    # Kiểm tra xem file checkpoint có tồn tại không
    if os.path.exists(in_dir):
        ckpt = torch.load(in_dir, map_location=device)
        
        # Load trạng thái model, optimizer, scheduler
        model.load_state_dict(ckpt["model_state"])
        if "optimizer_state" in ckpt:
            optimizer.load_state_dict(ckpt["optimizer_state"])
        if "scheduler_state" in ckpt:
            scheduler.load_state_dict(ckpt["scheduler_state"])
        print(f"[INFO] Loaded existing model from {ckpt_path}")

        # Load lịch sử training cũ
        if os.path.exists(history_dir):
            with open(history_dir, "r") as f:
                history = json.load(f)
            best_val = max(history["val_acc"]) / 100
            start_epoch = len(history["val_acc"])
            best_epoch = start_epoch - 1
            print(f"[INFO] Resuming training from epoch {start_epoch}, best val acc = {best_val*100:.2f}%")

    # Tạo danh sách tên các nhãn để in báo cáo
    label_list = [train_ds.idx_to_label[i] for i in range(len(train_ds.idx_to_label))]

    # ============================================================
    # VÒNG LẶP TRAINING (TRAINING LOOP)
    # ============================================================
    for epoch in range(start_epoch, epochs):

        # --- A. Quá trình Train ---
        model.train()
        running_loss = 0.0
        y_true_train, y_pred_train = [], []
        
        # Thanh tiến trình cho tập train
        pbar = tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{epochs}", leave=False)

        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            
            # Xóa gradient cũ
            optimizer.zero_grad(set_to_none=True)
            
            # Huấn luyện với Mixed Precision (AMP)
            with amp.autocast(device_type="cuda", enabled=(device.type=="cuda")):
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            
            # Backward và cập nhật trọng số với Scaler
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0) # Gradient Clipping
            scaler.step(optimizer)
            scaler.update()

            # Tính toán loss và lưu kết quả dự đoán
            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(dim=1).detach().cpu().numpy()
            y_pred_train.extend(preds.tolist())
            y_true_train.extend(labels.detach().cpu().numpy().tolist())

        # Tính metrics cho tập train
        train_loss = running_loss / len(train_ds)
        train_acc, _ = compute_metrics(y_true_train, y_pred_train)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc * 100)

        # --- B. Quá trình Validation ---
        model.eval()
        running_val_loss = 0.0
        y_true, y_pred = [], []
        
        # Không tính gradient khi validation
        with torch.no_grad(), amp.autocast(device_type="cuda", enabled=(device.type=="cuda")):
            for imgs, labels in tqdm(val_loader, desc=f"Val Epoch {epoch+1}/{epochs}", leave=False):
                imgs, labels = imgs.to(device), labels.to(device)
                
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                
                running_val_loss += loss.item() * imgs.size(0)
                preds = outputs.argmax(dim=1).detach().cpu().numpy()
                y_pred.extend(preds.tolist())
                y_true.extend(labels.detach().cpu().numpy().tolist())

        # Tính metrics cho tập val
        val_loss = running_val_loss / len(val_ds)
        val_acc, cm = compute_metrics(y_true, y_pred, labels=list(range(len(train_ds.idx_to_label))))
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc * 100)

        # --- C. Logging định kỳ (Mỗi 5 epoch) ---
        if epoch % 5 == 0:
            print("Class distribution check:")
            # Đếm số lượng dự đoán so với thực tế
            pred_counts = {label_list[i]: (np.array(y_pred) == i).sum() for i in range(len(label_list))}
            true_counts = {label_list[i]: (np.array(y_true) == i).sum() for i in range(len(label_list))}
            
            for lbl in label_list:
                ratio = pred_counts[lbl] / max(1, true_counts[lbl])
                print(f"  {lbl:<15} pred={pred_counts[lbl]:<6} true={true_counts[lbl]:<6} ratio={ratio:.2f}x")
            
            # In báo cáo chi tiết (Precision, Recall, F1)
            report = classification_report(y_true, y_pred, target_names=label_list, digits=3)
            print("\nPer-class performance:")
            print(report)

        # Cập nhật Learning Rate
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]

        # In kết quả Epoch hiện tại
        print(f"\nEpoch {epoch+1}/{epochs}")
        print(f"LR: {current_lr:.2e}")
        print("=" * 70)
        print("Results:")
        print(f"  Train: loss={train_loss:.4f}, acc={train_acc*100:.2f}%")
        print(f"  Val:   loss={val_loss:.4f}, acc={val_acc*100:.2f}%")
        print()

        # --- D. Checkpoint & Early Stopping ---
        # Nếu kết quả tốt hơn tốt nhất trước đó -> Lưu model
        if val_acc > best_val + 1e-6:
            best_val = val_acc
            best_epoch = epoch
            ckpt = {
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "scheduler_state": scheduler.state_dict(),
                "label_to_idx": train_ds.label_to_idx
            }
            torch.save(ckpt, ckpt_path)
            print(f"[INFO] Saved best model (val_acc={val_acc*100:.2f}%)")
            no_improve = 0
        else:
            # Nếu không cải thiện -> Tăng biến đếm
            no_improve += 1
            if no_improve >= early_stop_patience:
                print(f"[INFO] Early stopping at epoch {epoch+1}, best epoch {best_epoch+1}")
                # Lưu trạng thái cuối cùng trước khi dừng
                torch.save({
                    "epoch": epoch,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "scheduler_state": scheduler.state_dict(),
                    "label_to_idx": train_ds.label_to_idx
                }, final_ckpt_path)
                break

        # Lưu history vào file JSON
        with open(history_path, "w") as f:
            json.dump(history, f, indent=2)

        # Nếu là epoch cuối cùng -> Lưu model
        if epoch == epochs:
            torch.save({
                "epoch": epoch,
                "model_state": model.state_dict(),
                "optimizer_state": optimizer.state_dict(),
                "scheduler_state": scheduler.state_dict(),
                "label_to_idx": train_ds.label_to_idx
            }, final_ckpt_path)

    # --- 7. Vẽ biểu đồ (Plotting) ---
    plt.figure(figsize=(10,4))
    
    # Biểu đồ Loss
    plt.subplot(1,2,1)
    plt.plot(history["train_loss"], label="train_loss")
    plt.plot(history["val_loss"], label="val_loss")
    plt.legend(); plt.title("Loss")
    
    # Biểu đồ Accuracy
    plt.subplot(1,2,2)
    plt.plot(history["train_acc"], label="train_acc")
    plt.plot(history["val_acc"], label="val_acc")
    plt.legend(); plt.title("Accuracy")
    
    plt.tight_layout()
    plt.show()

    print(f"Best val acc: {best_val*100:.2f}% at epoch {best_epoch+1}")
    
    # Trả về model, history và danh sách nhãn
    return model, history, train_ds.idx_to_label

In [ ]:
if __name__ == "__main__":
        
    model, history, idx2label = train_and_evaluate(in_dir="/kaggle/input/emotion-model/pytorch/default/1/best_emotion.pth",
                                                   out_dir="checkpoints",
                                                   history_dir="/kaggle/input/emotion-model/pytorch/default/1/history.json",
                                                   img_size=112,
                                                   epochs=300,
                                                   lr=3e-4)


In [ ]:
history_dir="/kaggle/input/emotion-model/pytorch/default/1/history.json"
if os.path.exists(history_dir):
    with open(history_dir, "r") as f:
        history = json.load(f)
    best_val = max(history["val_acc"]) / 100
    start_epoch = len(history["val_acc"])
    best_epoch = start_epoch - 1

# --- Plot ---
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history["train_loss"], label="train_loss")
plt.plot(history["val_loss"], label="val_loss")
plt.legend(); plt.title("Loss")
plt.subplot(1,2,2)
plt.plot(history["train_acc"], label="train_acc")
plt.plot(history["val_acc"], label="val_acc")
plt.legend(); plt.title("Accuracy")
plt.tight_layout()
plt.show()

# Test predict

In [ ]:
def predict_from_url(model, url, transform, idx_to_label, device=None, show_image=True, top_k=None):
    """
    Dự đoán cảm xúc từ URL ảnh.
    Hiển thị top xác suất và biểu đồ cho toàn bộ lớp.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device

    try:
        # --- Gửi request ---
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # --- Kiểm tra xem có phải là ảnh không ---
        content_type = response.headers.get("Content-Type", "")
        if "image" not in content_type:
            raise ValueError(f"URL không trả về ảnh. Content-Type: {content_type}")

        # --- Mở ảnh ---
        # BytesIO giúp đọc dữ liệu binary từ RAM như một file
        image = Image.open(BytesIO(response.content)).convert("RGB")

    except (UnidentifiedImageError, ValueError) as e:
        print(f"[ERROR] Không thể mở ảnh từ URL: {url}\nLý do: {e}")
        return None, None
    except Exception as e:
        print(f"[ERROR] Lỗi khi tải ảnh: {e}")
        return None, None

    # --- Hiển thị ảnh ---
    if show_image:
        plt.imshow(image)
        plt.axis("off")
        plt.title("Input Image")
        plt.show()

    # --- Tiền xử lý (Transform) ---
    # unsqueeze(0) để tạo batch size = 1: [C, H, W] -> [1, C, H, W]
    img_tensor = transform(image).unsqueeze(0).to(device)

    # --- Dự đoán ---
    with torch.no_grad(): # Không tính gradient
        outputs = model(img_tensor)
        # Softmax để chuyển output thành xác suất (0-1)
        probs = F.softmax(outputs, dim=1).cpu().numpy()[0]

    # --- Xử lý kết quả ---
    emotions = [idx_to_label[i] for i in range(len(probs))]
    sorted_idx = probs.argsort()[::-1]  # Sắp xếp index theo xác suất giảm dần
    top_k = top_k or len(emotions) # Nếu không chỉ định top_k thì lấy hết

    print("Top dự đoán:")
    for i in range(top_k):
        lbl = emotions[sorted_idx[i]]
        conf = probs[sorted_idx[i]] * 100
        print(f"  {i+1}. {lbl:10s} : {conf:.2f}%")

    # --- Biểu đồ xác suất ---
    plt.figure(figsize=(8, 4))
    # Đảo ngược list để class có xác suất cao nhất nằm trên cùng biểu đồ ngang
    plt.barh([emotions[i] for i in sorted_idx[::-1]],
             [probs[i]*100 for i in sorted_idx[::-1]],
             color="skyblue")
    plt.xlabel("Probability (%)")
    plt.title("Emotion Probabilities")
    plt.tight_layout()
    plt.show()

    # --- Trả về nhãn cao nhất ---
    pred_label = emotions[sorted_idx[0]]
    pred_conf = probs[sorted_idx[0]]
    return pred_label, pred_conf


In [ ]:
def load_model_for_inference(ckpt_path, device=None):
    """
    Load trọng số model từ file checkpoint   
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device
    
    # Load file checkpoint
    print(f"[INFO] Loading checkpoint from: {ckpt_path}")
    ckpt = torch.load(ckpt_path, map_location=device)
    
    # Khôi phục dictionary nhãn
    label_to_idx = ckpt["label_to_idx"]
    idx_to_label = {v: k for k, v in label_to_idx.items()}

    # Khởi tạo kiến trúc model
    model = EmotionResNet(num_classes=len(label_to_idx))
    
    state_dict = ckpt["model_state"]
    
    # Kiểm tra xem key đầu tiên có chứa 'module.'(DataParallel)
    if list(state_dict.keys())[0].startswith('module.'):
        print("[INFO] Detected DataParallel checkpoint. Removing 'module.' prefix...")
        # Tạo state_dict mới bằng cách bỏ chữ 'module.' ở đầu mỗi key
        new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
        model.load_state_dict(new_state_dict)
    else:
        # Nếu không có 'module.', load bình thường
        model.load_state_dict(state_dict)

    model.to(device)
    model.eval()

    print(f"[INFO] Model loaded successfully!")
    return model, idx_to_label

if __name__ == "__main__":
    # Cấu hình thiết bị
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Đường dẫn checkpoint
    ckpt_path = "/kaggle/input/emotion-model/pytorch/default/1/best_emotion.pth"

    # 1. Load model
    if os.path.exists(ckpt_path):
        model, idx_to_label = load_model_for_inference(ckpt_path, device)

        # 2. Lấy transform
        _, val_t = get_transforms(img_size=112)

        # 3. Link ảnh test
        url = "https://t4.ftcdn.net/jpg/00/68/69/59/360_F_68695981_GuWIHWfB0l5wJ2al8rv4xZRUqUtwIo2P.jpg"

        # 4. Thực hiện dự đoán
        predict_from_url(model, url, val_t, idx_to_label, device)
    else:
        print(f"[ERROR] Checkpoint not found at: {ckpt_path}")

# Dowload checkpoints.zip

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
    
download_file('/kaggle/working/checkpoints', 'checkpoints') 